In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import torch

from torch.utils.tensorboard import SummaryWriter
from utils.nn_data_classifier import load_data, Classifier
from utils.preprocess import preprocess, RNNDataset

writer = SummaryWriter(log_dir='logs')



Load the dataset

In [ ]:
historical_data = load_data()

historical_data['minute'] = historical_data.Day.dt.minute
historical_data = historical_data[historical_data.minute == 0]
historical_data.drop('minute', axis=1, inplace=True)

historical_data

Classify data

In [ ]:
time_outlook = 3
classi = Classifier(historical_data)
classified_data = classi.classify_data_strict_time(time_outlook=time_outlook)
classified_data

Sort data

In [ ]:
data = classified_data.sort_values('Day').reset_index(drop=True)
data

In [ ]:
SEQ_LEN = 24

In [ ]:
processor = preprocess(sequence_length=SEQ_LEN)
training, validation, testing  = processor.preprocess(dataframe=data, validation_size=0.2)

In [ ]:
from torch import nn
class RNN_module(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, num_layers):
        super(RNN_module, self).__init__()
        self._num_layers = num_layers
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.lstm = nn.LSTM(input_size = self._input_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)
        self.fc = nn.Linear(in_features=self._hidden_size, out_features= self._output_size)

    def __str__(self):
        return f"RNN LSTM Model w/ {self._input_size} features and {self._num_layers} layers and {self._hidden_size} of hidden size"

    def forward(self, input):
        h_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)
        c_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)

        lstm_output, (h_n, c_n) = self.lstm(input)

        pred = self.fc(lstm_output[:, -1, :])
        
        return pred

In [ ]:
from torch.utils.data import DataLoader

learning_rate = 0.01
dim_size = training._features.size(dim=-1)
hidden_size = 60
batch_size = 64
number_of_classes = training[1][1].shape[0]

train_dataloader = DataLoader(training, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(validation, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testing, batch_size = batch_size, shuffle = False)

model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

In [ ]:
labels = training._labels
classes, _ = torch.sort(torch.unique(labels, dim = 0))
weights = torch.zeros((classes.shape[0],))

total = len(data)

for idx, _class in enumerate(classes):
    freq = len(data[data.Classification == _class.item()])
    weights[idx] += 1 - freq/total

weights

In [ ]:
# labels = training._labels
# classes = torch.unique(labels, dim = 0)

# freq = {}
# for _class in classes:
#     freq[_class.item()] = 0

# weights = torch.zeros((classes.shape[0],))
# total = torch.tensor(len(training))

# for input in range(training._beg, training._end):
#     _, output = torch.max(training[input][1], 0)
#     freq[output.item()] += 1


# for idx, _class in enumerate(classes):
#     weights[idx] +=  1 - freq[_class.item()]/total

# weights

In [ ]:
criterion = nn.CrossEntropyLoss(weight=weights)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
CHECKPOINT_PATH = './models_parameters/LSTM/checkpoints_2/'
BEST_PATH = './models_parameters/LSTM/best_model.pth'

def epoch_training(model, train_dataloader, epoch, total_epochs, optimizer):
    running_correct = 0
    running_loss = 0.0
    n_of_steps = len(train_dataloader)

    for current_batch, (sequence, label) in enumerate(train_dataloader):
        #forward: we are calculating the loss given the parameters
        outputs = model(sequence)
        loss = criterion(input=outputs, target = label.float())

        #backward: lets update the parameters given the current loss
        optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
        loss.backward() #computates the bwrd-prop gradient for each model parameter
        optimizer.step() #updates the model current parameter using the gradients.

        predictions = torch.argmax(outputs, 1)
        correct = torch.argmax(label, 1)

        running_correct += (predictions == correct).sum().item()
        running_loss += loss.item()

        if (current_batch + 1) % 200 == 0:
            print(f"epoch {epoch+1}/{total_epochs}, current step(batch): {current_batch+1}/{n_of_steps}, loss = {loss.item():.4f} ")
            writer.add_scalar('training loss: ', running_loss/200, epoch * n_of_steps + current_batch)
            writer.add_scalar('accuracy: ', running_correct/200, epoch * n_of_steps + current_batch)
            running_loss = 0.0
            running_correct = 0

    writer.add_scalar('Epoch loss: ', loss, epoch + 1)


def epoch_validate(model, validation_dataloader, epoch, total_epochs):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0
        
        for current_batch, (sequence, label) in enumerate(validation_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)
            
            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

        acc = 100.0 * n_corrects / n_samples

        print(f"epoch {epoch+1}/{total_epochs} accuracy: {acc}")
        writer.add_scalar('Validation Accuracy: ', acc, epoch+1)

    return acc


def train_loop(model: RNN_module, train_dataloader: DataLoader, validation_dataloader: DataLoader, epochs: int):
    
    max_accuracy = 0
    is_best = False
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        epoch_training(model, train_dataloader, epoch, epochs, optimizer)

        accuracy = epoch_validate(model, validation_dataloader, epoch, epochs)

        if accuracy > max_accuracy:
            is_best = True
            max_accuracy = accuracy
        else:
            is_best = False
        
        checkpoint = {
            'epoch': epoch+1,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict()
        }

        if is_best:
            torch.save(checkpoint, BEST_PATH)
        
        torch.save(checkpoint, CHECKPOINT_PATH+f'model_{epoch+1}.pth')

In [ ]:
def test_loop(test_dataloader: DataLoader, model: nn.Module):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(test_dataloader):
            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = torch.argmax(outputs, 1)

            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

            if (current_batch + 1) % 200 == 0:
                print(f"test batch: {current_batch+1}/{len(test_dataloader)}, current accuracy: {100 * n_corrects / n_samples}")

        acc = 100.0 * n_corrects / n_samples
        print(f"final test accuracy: {acc}")


In [ ]:
c = model.state_dict()

In [ ]:
epochs = 10
train_loop(model, train_dataloader=train_dataloader, validation_dataloader = validation_dataloader, epochs=epochs)
test_loop(test_dataloader=test_dataloader, model=model)

Using best model in validation

In [ ]:
best_model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

checkpoint = torch.load(BEST_PATH)
print(f'Model type: {best_model}')
print(f'Best performing model found at {checkpoint["epoch"]}ºepoch')

best_model.load_state_dict(state_dict=checkpoint['model_state'], strict=True)
best_model.eval()

test_loop(test_dataloader=test_dataloader, model=best_model)

In [ ]:
checkpoint = torch.load('./models_parameters/LSTM/checkpoints_2/model_1.pth')

a = checkpoint['model_state']

checkpoint = torch.load('./models_parameters/LSTM/checkpoints_2/model_69.pth')

b = checkpoint['model_state']

In [ ]:
writer.close()